📝 **Author:** Amirhossein Heydari - 📧 **Email:** <amirhosseinheydari78@gmail.com> - 📍 **Origin:** [mr-pylin/pytorch-workshop](https://github.com/mr-pylin/pytorch-workshop)

---


**Table of contents**<a id='toc0_'></a>    
- [Dependencies](#toc1_)    
- [DenseNet](#toc2_)    
  - [Custom DenseNet](#toc2_1_)    
    - [DenseNet-121](#toc2_1_1_)    
    - [DenseNet-161](#toc2_1_2_)    
    - [DenseNet-169](#toc2_1_3_)    
    - [DenseNet-201](#toc2_1_4_)    
  - [PyTorch DenseNet](#toc2_2_)    
    - [DenseNet-121](#toc2_2_1_)    
    - [DenseNet-161](#toc2_2_2_)    
    - [DenseNet-169](#toc2_2_3_)    
    - [DenseNet-201](#toc2_2_4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Dependencies](#toc0_)

- torchvision models:
  - class
    - brings in the model class directly
    - Allows more control and customization since you are dealing directly with the class. You can override methods, customize initialization, etc.
  - function
    - This import brings in a function that returns an instance of the model
    - Easier and quicker to use, especially for standard models
- [pytorch.org/vision/stable/models.html](https://pytorch.org/vision/stable/models.html)


In [ ]:
import torch
from torch import nn
from torch.functional import F
from torchinfo import summary
from torchvision.models import DenseNet, densenet121, densenet161, densenet169, densenet201

# <a id='toc2_'></a>[DenseNet](#toc0_)

- Densely Connected Convolutional Network (DenseNet), developed in 2017 by [Gao Huang](https://scholar.google.com.hk/citations?user=-P9LwcgAAAAJ&hl) and collaborators from [Cornell University](https://www.cornell.edu/) and [Tsinghua University](https://www.tsinghua.edu.cn/en/)
- It is based on the [Densely Connected Convolutional Networks](https://openaccess.thecvf.com/content_cvpr_2017/papers/Huang_Densely_Connected_Convolutional_CVPR_2017_paper.pdf) paper
- It was trained on the [ImageNet](https://www.image-net.org/) dataset (first resized to 256x256 then center cropped to 224x224) [[ImageNet viewer](https://navigu.net/#imagenet)]
- Known for its innovative use of `dense connections` where each layer receives the feature maps of all preceding layers, enhancing gradient flow and feature reuse
- It comes in several variants, primarily `DenseNet-121`, `DenseNet-161`, `DenseNet-169` and `DenseNet-201`, indicating the depth and complexity of the network
- Achieved high performance in various benchmarks and demonstrated significant parameter efficiency and feature reuse due to its dense connectivity

<figure style="text-align: center;">
  <img src="../../../assets/images/original/cnn/architectures/densenet.svg" alt="densenet-architecture.svg" style="width: 100%;">
  <figcaption>DenseNet Architecture</figcaption>
</figure>


## <a id='toc2_1_'></a>[Custom DenseNet](#toc0_)

- `Softmax` is missing due to internal implementation of `LogSoftmax` in the `CrossEntropyLoss` function.


In [ ]:
class DenseLayer(nn.Module):
    def __init__(self, in_channels, growth_rate) -> None:
        super().__init__()
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.conv1 = nn.Conv2d(in_channels, 4 * growth_rate, kernel_size=1, bias=False)
        self.bn2 = nn.BatchNorm2d(4 * growth_rate)
        self.conv2 = nn.Conv2d(4 * growth_rate, growth_rate, kernel_size=3, padding=1, bias=False)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out = self.conv1(F.relu(self.bn1(x)))
        out = self.conv2(F.relu(self.bn2(out)))
        out = torch.cat([x, out], 1)
        return out

In [ ]:
class DenseBlock(nn.Module):
    def __init__(self, num_layers, in_channels, growth_rate) -> None:
        super().__init__()
        layers = []
        for i in range(num_layers):
            layers.append(DenseLayer(in_channels + i * growth_rate, growth_rate))
        self.block = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.block(x)

In [ ]:
class TransitionLayer(nn.Module):
    def __init__(self, in_channels, out_channels) -> None:
        super().__init__()
        self.bn = nn.BatchNorm2d(in_channels)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.pool = nn.AvgPool2d(2, stride=2)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out = self.conv(F.relu(self.bn(x)))
        out = self.pool(out)
        return out

In [ ]:
class CustomDenseNet(nn.Module):
    def __init__(self, num_layers_per_block, growth_rate, num_classes=1000) -> None:
        super().__init__()
        self.growth_rate = growth_rate
        num_channels = 2 * growth_rate

        # densenet-121, densenet-169, densenet-201 : 3x224x224 -> 64x112x112
        # densenet-161                             : 3x224x224 -> 96x112x112
        self.conv1 = nn.Conv2d(3, num_channels, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.relu = nn.ReLU(inplace=True)

        # densenet-121, densenet-169, densenet-201 : 64x112x112 -> 64x56x56
        # densenet-161                             : 96x112x112 -> 96x56x56
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # densenet-121 : 64x56x56 -> 1024x7x7
        # densenet-161 : 96x56x56 -> 2208x7x7
        # densenet-169 : 64x56x56 -> 1664x7x7
        # densenet-201 : 64x56x56 -> 1920x7x7
        blocks = []
        for i in range(len(num_layers_per_block)):
            blocks.append(DenseBlock(num_layers_per_block[i], num_channels, growth_rate))
            num_channels += num_layers_per_block[i] * growth_rate
            if i != len(num_layers_per_block) - 1:
                blocks.append(TransitionLayer(num_channels, num_channels // 2))
                num_channels = num_channels // 2

        self.blocks = nn.Sequential(*blocks)
        self.bn2 = nn.BatchNorm2d(num_channels)

        # densenet-121 : 1024x7x7 -> 1024x1x1
        # densenet-161 : 2208x7x7 -> 2208x1x1
        # densenet-169 : 1664x7x7 -> 1664x1x1
        # densenet-201 : 1920x7x7 -> 1920x1x1
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        # densenet-121 : 1024 -> 1000
        # densenet-161 : 2208 -> 1000
        # densenet-169 : 1664 -> 1000
        # densenet-201 : 1920 -> 1000
        self.fc = nn.Linear(num_channels, num_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # feature extractor
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.pool1(out)
        out = self.blocks(out)
        out = self.bn2(out)
        out = self.relu(out)

        # adaptive average pooling
        out = self.avgpool(out)

        # flatten:
        # densenet-121 : 1024x1x1 -> 1024
        # densenet-161 : 1536x1x1 -> 1536
        # densenet-169 : 1664x1x1 -> 1664
        # densenet-201 : 1920x1x1 -> 1920
        out = torch.flatten(out, 1)

        # classifier
        out = self.fc(out)
        return out

### <a id='toc2_1_1_'></a>[DenseNet-121](#toc0_)


In [ ]:
densenet_121_1 = CustomDenseNet(num_layers_per_block=[6, 12, 24, 16], growth_rate=32)
densenet_121_1

In [ ]:
summary(densenet_121_1, (1, 3, 224, 224), device="cpu")

### <a id='toc2_1_2_'></a>[DenseNet-161](#toc0_)


In [ ]:
densenet_161_1 = CustomDenseNet(num_layers_per_block=[6, 12, 36, 24], growth_rate=48)
densenet_161_1

In [ ]:
summary(densenet_161_1, (1, 3, 224, 224), device="cpu")

### <a id='toc2_1_3_'></a>[DenseNet-169](#toc0_)


In [ ]:
densenet_169_1 = CustomDenseNet(num_layers_per_block=[6, 12, 32, 32], growth_rate=32)
densenet_169_1

In [ ]:
summary(densenet_169_1, (1, 3, 224, 224), device="cpu")

### <a id='toc2_1_4_'></a>[DenseNet-201](#toc0_)


In [ ]:
densenet_201_1 = CustomDenseNet(num_layers_per_block=[6, 12, 48, 32], growth_rate=32)
densenet_201_1

In [ ]:
summary(densenet_201_1, (1, 3, 224, 224), device="cpu")

## <a id='toc2_2_'></a>[PyTorch DenseNet](#toc0_)

- DenseNet is available in PyTorch: [pytorch.org/vision/main/models/densenet.html](https://pytorch.org/vision/main/models/densenet.html)


### <a id='toc2_2_1_'></a>[DenseNet-121](#toc0_)


In [ ]:
densenet_121_2 = densenet121()
densenet_121_2

In [ ]:
summary(densenet_121_2, (1, 3, 224, 224), device="cpu")

### <a id='toc2_2_2_'></a>[DenseNet-161](#toc0_)


In [ ]:
densenet_161_2 = densenet161()
densenet_161_2

In [ ]:
summary(densenet_161_2, (1, 3, 224, 224), device="cpu")

### <a id='toc2_2_3_'></a>[DenseNet-169](#toc0_)


In [ ]:
densenet_169_2 = densenet169()
densenet_169_2

In [ ]:
summary(densenet_169_2, (1, 3, 224, 224), device="cpu")

### <a id='toc2_2_4_'></a>[DenseNet-201](#toc0_)


In [ ]:
densenet_201_2 = densenet201()
densenet_201_2

In [ ]:
summary(densenet_201_2, (1, 3, 224, 224), device="cpu")